In [0]:
# Defining the Lapse and no Lapse:
# Each month, some policies are either paid or not paid. For example, in the month of Jan 2018, there are 100 policies paid and 50 policies not paid. 
# The policies that are paid are called Lapse, and the policies that are not paid are called No Lapse. 
# Step:
# 1. Define the time window to make a model. Let's start with Jan 2018.
# 2. Collect the active policies and the Paid_to_Date is in the time window, here Jan 2018.
# 3. From the policies in step 2, check:
#    a. The new paid_to_date. If there is no newer paid_to_date, then the policy is Lapse.

In [0]:
%sql
USE CATALOG lapse_scoring_dev;

WITH jan18 AS (
  SELECT *
  FROM `03_gold`.chandra_union_gold
  WHERE Paid_to_Date >= DATE '2018-01-01'
    AND Paid_to_Date <  DATE '2018-02-01'
    AND Payment_Frequency <> "0"
),
all_max_ptd AS (
  SELECT
    Contract_Number,
    MAX(Paid_to_Date) AS max_paid_to_date
  FROM `03_gold`.chandra_union_gold
  GROUP BY Contract_Number
)
SELECT
  a.*,
  m.max_paid_to_date AS latest_paid_to_date_overall,
  CASE
    WHEN a.Paid_to_Date <  m.max_paid_to_date THEN 0   -- masih ada pembayaran yang lebih baru
    WHEN a.Paid_to_Date >= m.max_paid_to_date THEN 1   -- ini sudah paling baru (atau sama)
    ELSE NULL                                          -- jika max atau current null
  END AS `Lapse Flag`
FROM jan18 a
LEFT JOIN all_max_ptd m
  ON a.Contract_Number = m.Contract_Number
ORDER BY a.Contract_Number;
